In [132]:
import torch
import pandas as pd
import torch.nn as nn
from pandas import DataFrame


In [133]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col=0)

In [134]:
object_columns = train_df.dtypes[train_df.dtypes==object].index.to_list()
numeric_columns = train_df.dtypes[train_df.dtypes!=object].index.to_list()

In [135]:
train_df[numeric_columns] = train_df[numeric_columns].fillna(0)
train_df[object_columns] = train_df[object_columns].fillna('NA')

In [136]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in object_columns:
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])
    
train_df[object_columns].head(5)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,3,1,1,3,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
2,3,1,1,3,3,0,2,0,24,1,...,1,2,5,5,2,3,4,1,8,4
3,3,1,1,0,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
4,3,1,1,0,3,0,0,0,6,2,...,5,3,5,5,2,3,4,1,8,0
5,3,1,1,0,3,0,2,0,15,2,...,1,2,5,5,2,3,4,1,8,4


In [137]:
train_df = train_df.astype(float)

In [138]:
means, maxs, mins = dict(), dict(), dict()
for col in train_df:
    means[col] = train_df[col].mean()
    maxs[col] = train_df[col].max()
    mins[col] = train_df[col].min()
    
train_df = (train_df - train_df.mean()) / (train_df.max() - train_df.min())
train_df.describe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1460.000000,1.460000e+03,...,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03
mean,-2.433366e-17,-4.197556e-17,1.155849e-17,-3.193792e-18,3.862968e-17,1.216683e-18,2.250863e-17,-3.528380e-17,0.000000,4.866731e-17,...,-9.125121e-19,3.361086e-17,2.007527e-17,-1.916275e-17,1.901067e-19,2.615868e-17,1.183589e-14,2.418157e-17,2.555034e-17,1.460019e-17
std,2.488269e-01,1.580044e-01,1.107486e-01,4.665342e-02,6.399614e-02,1.248334e-01,4.697187e-01,2.358886e-01,0.026171,4.056586e-01,...,5.444080e-02,4.690095e-02,2.707280e-01,9.302551e-02,3.200794e-02,2.457842e-01,3.320238e-01,1.940125e-01,2.201709e-01,1.103215e-01
min,-2.170427e-01,-7.571918e-01,-1.841000e-01,-4.308036e-02,-9.958904e-01,-4.969178e-01,-6.474886e-01,-9.257991e-01,-0.000685,-7.547945e-01,...,-3.738352e-03,-9.970320e-01,-8.761986e-01,-2.672945e-01,-2.805745e-03,-4.838107e-01,-4.539384e-01,-9.391267e-01,-7.541096e-01,-2.027791e-01
25%,-2.170427e-01,-7.191781e-03,-4.991466e-02,-1.385089e-02,4.109589e-03,3.082192e-03,-6.474886e-01,7.420091e-02,-0.000685,-2.547945e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,-1.201743e-01,-2.039384e-01,6.087329e-02,4.589041e-02,-7.074878e-02
50%,-4.057212e-02,-7.191781e-03,1.717799e-02,-4.853248e-03,4.109589e-03,3.082192e-03,3.525114e-01,7.420091e-02,-0.000685,2.452055e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,-2.926526e-02,4.606164e-02,6.087329e-02,4.589041e-02,-2.488709e-02
75%,7.707494e-02,-7.191781e-03,6.829621e-02,5.069863e-03,4.109589e-03,3.082192e-03,3.525114e-01,7.420091e-02,-0.000685,2.452055e-01,...,-3.738352e-03,2.968037e-03,1.238014e-01,-1.729452e-02,-2.805745e-03,1.525529e-01,2.960616e-01,6.087329e-02,4.589041e-02,4.593640e-02
max,7.829573e-01,2.428082e-01,8.159000e-01,9.569196e-01,4.109589e-03,5.030822e-01,3.525114e-01,7.420091e-02,0.999315,2.452055e-01,...,9.962616e-01,2.968037e-03,1.238014e-01,7.327055e-01,9.971943e-01,5.161893e-01,5.460616e-01,6.087329e-02,2.458904e-01,7.972209e-01


In [139]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.data = df.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index, :-1])
        y = torch.tensor(self.data[index, -1:])
        return x, y

In [140]:
dataset = CustomDataset(train_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [141]:
class Net(nn.Module):
    def __init__(self, d_in, h1, h2, h3, d_out):
        super(Net, self).__init__()
        
        self.linear1 = nn.Linear(d_in, h1, dtype=float)
        self.linear2 = nn.Linear(h1, h2, dtype=float)
        self.linear3 = nn.Linear(h2, h3, dtype=float)
        self.linear4 = nn.Linear(h3, d_out, dtype=float)
        
    def forward(self, x):                
        y_pred = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(y_pred).clamp(min=0)
        y_pred = self.linear3(y_pred).clamp(min=0)
        y_pred = self.linear4(y_pred)
        return y_pred

In [142]:
h1, h2, h3 = 500, 1000, 200
d_in, d_out = train_df.shape[1] - 1, 1

In [143]:
model1 = Net(d_in, h1, h2, h3, d_out)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model1.parameters(), lr=2e-2)

In [144]:
epoch_count = 1000
for epoch in range(epoch_count):
    loss_sum = 0.0    
    for batch in dataloader:
        x_tensor, y_tensor = batch
        y_pred = model1(x_tensor)        
        loss = criterion(y_pred, y_tensor)            
        loss_sum += loss.item()                
        
        if torch.isnan(loss):
            break

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(epoch, loss_sum)

0 12.342587030498157
1 9.294275658192209
2 5.556439020424439
3 6.893353257722499
4 4.961294155994512
5 4.102281382686606
6 3.6857843100938434
7 3.3781384315293503
8 3.131261021533593
9 3.416101094230709
10 2.74932933544311
11 2.8914037264757146
12 2.3513552765483112
13 3.348955583670812
14 2.3343807174952063
15 3.2017491296023284
16 2.1411173479091747
17 2.135030689872212
18 2.0967999688471775
19 2.014611633946096
20 2.090000031467964
21 2.390722248800448
22 2.5445370688115356
23 1.795885411115554
24 1.5630739418591573
25 1.727586027495008
26 2.098452370288207
27 1.4103580597414203
28 2.4824681643005464
29 1.528591095168804
30 1.5408393255947554
31 1.0526127338096791
32 1.977423887037532
33 1.3583354218449126
34 0.9903317117744376
35 2.033721143132245
36 1.377794903831814
37 0.9737398420690392
38 0.8775790647745759
39 1.6206325624622457
40 1.0091503506485224
41 0.9230949474645738
42 1.0874361143432787
43 2.0741820754350404
44 1.1041311328619647
45 0.8914714269056172
46 1.05809718861081

In [145]:
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col=0)

In [146]:
numeric_columns.remove('SalePrice')
test_df[numeric_columns] = test_df[numeric_columns].fillna(0)
test_df[object_columns] = test_df[object_columns].fillna('NA')

In [147]:
for col in object_columns:
    le.fit(test_df[col])
    test_df[col] = le.transform(test_df[col])
    
test_df[object_columns].head(5)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,3,1,1,3,3,0,4,0,12,1,...,1,3,4,5,2,2,2,1,9,4
1462,4,1,1,0,3,0,0,0,12,2,...,1,3,4,5,2,2,4,0,9,4
1463,4,1,1,0,3,0,4,0,8,2,...,1,0,4,5,2,2,2,1,9,4
1464,4,1,1,0,3,0,4,0,8,2,...,1,0,4,5,2,2,4,1,9,4
1465,4,1,1,0,1,0,4,0,22,2,...,1,2,4,5,2,2,4,1,9,4


In [148]:
train_df = train_df.astype(float)

In [149]:
for col in test_df:
    test_df[col] = (test_df[col] - means[col]) / (maxs[col] - mins[col])
    
test_df.describe()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,...,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,0.002830,0.232870,0.000917,-0.003261,-0.000003,-0.008227,0.003414,-0.000279,0.000686,0.018338,...,0.004174,-0.001375,-0.331508,-0.005568,-0.001702,0.000947,-0.019794,-0.011512,0.112535,0.003396
std,0.251452,0.224114,0.103086,0.023163,0.064018,0.134978,0.470250,0.233780,0.037012,0.396449,...,0.117937,0.041317,0.026164,0.275010,0.088648,0.040697,0.247494,0.325435,0.230595,0.211087
min,-0.217043,-0.757192,-0.184100,-0.042286,-0.995890,-0.496918,-0.647489,-0.925799,-0.000685,-0.754795,...,-0.031377,-0.003738,-0.997032,-0.876199,-0.267295,-0.002806,-0.483811,-0.453938,-0.939127,-0.754110
25%,-0.217043,0.242808,-0.043525,-0.014610,0.004110,0.003082,-0.647489,0.074201,-0.000685,-0.254795,...,-0.031377,-0.003738,-0.330365,0.123801,-0.017295,-0.002806,-0.211083,-0.203938,0.185873,0.045890
50%,-0.040572,0.242808,0.017178,-0.005225,0.004110,0.003082,0.352511,0.074201,-0.000685,0.245205,...,-0.031377,-0.003738,-0.330365,0.123801,-0.017295,-0.002806,-0.029265,0.046062,0.185873,0.045890
75%,0.077075,0.242808,0.065101,0.004677,0.004110,0.003082,0.352511,0.074201,-0.000685,0.245205,...,-0.031377,-0.003738,-0.330365,0.123801,-0.017295,-0.002806,0.152553,0.296062,0.185873,0.045890
max,0.782957,0.492808,0.454878,0.215397,0.004110,0.503082,0.352511,0.074201,0.999315,0.245205,...,1.168623,1.080272,-0.330365,0.123801,0.482705,1.093968,0.516189,0.546062,0.185873,0.245890


In [150]:
x_tensor = torch.tensor(test_df.iloc[:, :].values, dtype=float)
x_tensor.shape

torch.Size([1459, 79])

In [151]:
y_tensor = model1(x_tensor)
y_tensor.shape

torch.Size([1459, 1])

In [152]:
with torch.no_grad():
    outputs = model1(x_tensor)    
    result = pd.DataFrame({'Id': test_df.index, 'SalePrice': outputs.reshape(outputs.shape[0])})
    result['SalePrice'] = result['SalePrice'] * (maxs['SalePrice'] - mins['SalePrice']) + means['SalePrice']    
    result.to_csv('sub.csv', index=False)
    